In [1]:
import fsspec
import os
from ipfsspec.asyn import AsyncIPFSFileSystem
from fsspec import register_implementation
import asyncio
import io
import glob

# register_implementation(IPFSFileSystem.protocol, IPFSFileSystem)
register_implementation(AsyncIPFSFileSystem.protocol, AsyncIPFSFileSystem)

# with fsspec.open("ipfs://QmZ4tDuvesekSs4qM5ZBKpXiZGun7S2CYtEZRB3DYXkjGx", "r") as f:
#     print(f.read())
class fs:
    ipfs = fsspec.filesystem("ipfs")
    file = fsspec.filesystem("file")

Changed to local node


# Changing Gateways

In [2]:
fs.ipfs.change_gateway_type = 'public'

Changed to public node


In [3]:
fs.ipfs.gateway.gws

[(<ipfsspec.gateway.GatewayState at 0x7fb06c924e80>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fb06c924490>),
 (<ipfsspec.gateway.GatewayState at 0x7fb049f4a520>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fb06c924910>),
 (<ipfsspec.gateway.GatewayState at 0x7fb049f4a910>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fb06c9247f0>),
 (<ipfsspec.gateway.GatewayState at 0x7fb049f4a970>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fb06c924f40>)]

In [4]:
fs.ipfs.gateway.gws[0][1].url

'https://ipfs.io'

In [5]:
fs.ipfs.change_gateway_type = 'local'

Changed to local node


In [6]:
fs.ipfs.gateway.gws[0][1].url

'http://ipfs:8080'

# Local

## Put

#### Put File

In [7]:
# put in a file and store it into test folder 'fam'
put_file_res = fs.ipfs.put(path='test/data/input/yo.txt', rpath='/test_put_file')
fs.ipfs.expand_path('/test_put_file/yo.txt')

['/test_put_file/yo.txt']

#### Put Folder

In [8]:
# put a directory and store it into test folder
fs.ipfs.put(path='test/data/input', rpath='/test_put_folder', recursive=True, return_cid=False)
fs.ipfs.ls('/test_put_folder')

[{'name': '/test_put_folder/bruh.txt',
  'CID': 'QmT9N3Kpo2qg7anf1fvNDFnheSrHat4Xu5fRte7tNG2AYH',
  'type': 'file',
  'size': 13},
 {'name': '/test_put_folder/yo.txt',
  'CID': 'QmfVErUtH2gi9HB2Z9TNPzBySPBptSeJftYzoRiGaa2v4N',
  'type': 'file',
  'size': 10}]

## Cat

#### Cat File

In [9]:
# Given a file CID to cat
fs.ipfs.cat('QmfVErUtH2gi9HB2Z9TNPzBySPBptSeJftYzoRiGaa2v4N')

b'yooooooooo'

In [10]:
# Given a file mfs to cat
fs.ipfs.cat('/test_put_file/yo.txt')

b'yooooooooo'

#### Cat Folder

In [11]:
# Given a directory CID to cat
fs.ipfs.cat('Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7')

{'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/bruh.txt': b'bruuhhhhhhhhh',
 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/yo.txt': b'yofvervevdfvd'}

In [12]:
# Given a file mfs to cat
fs.ipfs.cat('/test_put_folder')

{'/test_put_folder/bruh.txt': b'bruuhhhhhhhhh',
 '/test_put_folder/yo.txt': b'yooooooooo'}

## Remove

#### Remove File

In [13]:
fs.ipfs.put(path='test/data/input/yo.txt', rpath='/test_rm_file' ,recursive=True, return_cid=False)
print('Before: ',fs.ipfs.ls('/test_rm_file', detail=False))
fs.ipfs.rm('/test_rm_file/yo.txt')
print('After: ',fs.ipfs.ls('/test_rm_file', detail=False))

Before:  ['/test_rm_file/yo.txt']
After:  []


#### Remove Folder

In [14]:
fs.ipfs.put(path='test/data', rpath='/test_rm_folder', recursive=True, return_cid=False)
print('Before: ',fs.ipfs.ls('/test_rm_folder', detail=False))
fs.ipfs.rm('/test_rm_folder')
print('After: ',fs.ipfs.ls('/test_rm_folder', detail=False))

Before:  ['/test_rm_folder/bruh.txt', '/test_rm_folder/dataset', '/test_rm_folder/input', '/test_rm_folder/model', '/test_rm_folder/output', '/test_rm_folder/yo.txt']
After:  []


## Get

#### Get Remote File - local CID Path

In [15]:
def test_get_local_cid(lpath = 'test/data/input/yo.txt', rpath='/test_get_file',out_lpath = 'test/data/output' ):
    fs.ipfs.change_gateway_type = 'local'
    if fs.file.exists(out_lpath):
        fs.file.rm(out_lpath, recursive=True)
    fs.ipfs.rm(rpath, recursive=True)
    cid = fs.ipfs.put(lpath=lpath, rpath=rpath)
    print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob(f'{out_lpath}/*')])
    fs.ipfs.get(rpath=cid, lpath=out_lpath,recursive=True, return_cid=False)
    print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob(f'{out_lpath}/*')])
    fs.ipfs.rm(rpath, recursive=True)

test_get_local_cid()

Changed to local node
Before:  []
After:  ['test/data/output/QmfVErUtH2gi9HB2Z9TNPzBySPBptSeJftYzoRiGaa2v4N']


#### Get Remote File - Public CID Path

In [16]:
fs.ipfs.info('QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9')

{'name': 'QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9',
 'size': 1139,
 'ETag': 'QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9',
 'type': 'file',
 'CID': 'QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9'}

In [17]:
def test_get_public_cid(cid='QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9', 
                        rpath='/test_get_file',
                        out_lpath = 'test/data/output', 
                        gateway_type=None):
    if gateway_type:
        fs.ipfs.change_gateway_type = 'local'
    if fs.file.exists(out_lpath):
        fs.file.rm(out_lpath, recursive=True)
    fs.ipfs.rm(rpath, recursive=True)
    print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob(f'{out_lpath}/*')])
    fs.ipfs.get(rpath=cid, lpath=out_lpath,recursive=True, return_cid=False)
    print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob(f'{out_lpath}/*')])
    fs.ipfs.rm(rpath, recursive=True)

test_get_public_cid()

Before:  []
After:  ['test/data/output/QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9']


#### Get Remote File - MFS Path

In [18]:
def test_get_mfs(lpath = 'test/data/input/yo.txt', rpath='/test_get_file',out_lpath = 'test/data/output' ):
    fs.ipfs.change_gateway_type = 'local'
    if fs.file.exists(out_lpath):
        fs.file.rm(out_lpath, recursive=True)
    fs.ipfs.rm(rpath, recursive=True)
    cid = fs.ipfs.put(lpath=lpath, rpath=rpath)
    print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob(f'{out_lpath}/*')])
    fs.ipfs.get(rpath=rpath, lpath=out_lpath,recursive=True, return_cid=False)
    print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob(f'{out_lpath}/*')])
    fs.ipfs.rm(rpath, recursive=True)

test_get_mfs()

Changed to local node
Before:  []
After:  ['test/data/output/yo.txt']


#### Get Remote Folder - MFS Path

In [19]:
if fs.file.exists('test/data/output'):
    fs.file.rm('test/data/output', recursive=True)
    
fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])
fs.ipfs.get(rpath='/test_get_folder', lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])


Before:  []
After:  ['test/data/output/bruh.txt', 'test/data/output/yo.txt']


#### Get Remote Folder - CID Path

In [20]:
if fs.file.exists('test/data/output'):
    fs.file.rm('test/data/output', recursive=True)
    
fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

fs.ipfs.get(rpath='Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7', lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

Before:  []
After:  ['test/data/output/bruh.txt', 'test/data/output/yo.txt']


# Public

In [21]:
import fsspec
import os
from ipfsspec.asyn import AsyncIPFSFileSystem
from fsspec import register_implementation
import asyncio
import io

# register_implementation(IPFSFileSystem.protocol, IPFSFileSystem)
register_implementation(AsyncIPFSFileSystem.protocol, AsyncIPFSFileSystem)

# with fsspec.open("ipfs://QmZ4tDuvesekSs4qM5ZBKpXiZGun7S2CYtEZRB3DYXkjGx", "r") as f:
#     print(f.read())
class fs:
    ipfs = fsspec.filesystem("ipfs")
    file = fsspec.filesystem("file")

In [22]:
fs.ipfs.change_gateway_type = 'public'

Changed to public node


## Cat

#### Cat File

In [23]:
# Given a file CID to cat
cid_file = 'QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9'
fs.ipfs.cat(cid_file)

b"About this archive\nThis is an archive of the Project Apollo Archive, a re-presentation of the public domain NASA-provided Apollo mission imagery as it was originally provided in its raw, high-resolution and unprocessed form by the Johnson Space Center on DVD-R and including from the center's Gateway to Astronaut Photography of Earth web site.\n\nDirectory names are base64 encoded names of each of the albums in the archive. This was done in order to handle characters in the album names that aren't valid in directory names.\n\nUpdating this archive\nUse the included apolloarchive.py script. Before running, you will need to `pip install flickrapi`. The script also needs to have an API key and secret specified.\n\nIf you don't already have one, you can apply for a non-commercial flickr API at https://www.flickr.com/services/apps/create/apply.\n\nUpdating the frontend\nThe frontend lives in the frontend directory and can be updated using the build_frontend_index.py script. This script wi

#### Cat Folder

In [24]:
fs.ipfs.ls('QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8')

[{'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21751523890_3b65b9f030_o.jpg',
  'CID': 'QmbgeZnTVXAhW2CS15cjo981P81kULbuUqJ672YEGZUqiM',
  'type': 'file',
  'size': 386602},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21751523890_3b65b9f030_o.jpg.xml',
  'CID': 'QmRfG6WQEBXGJge8EQVXbYfYBS25pmhLoaczd5CQxjcT4U',
  'type': 'file',
  'size': 3513},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21878909302_a369c0c4de_o.jpg',
  'CID': 'QmXfbZ7H946MeecTWZqcdWKnPwudcqcokTFctJ5LeqMDK3',
  'type': 'file',
  'size': 1130403},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21878909302_a369c0c4de_o.jpg.xml',
  'CID': 'QmWxfUFH6ka7P5kMfw3ti3FhRqBpbkgFFY3NyaCZYxbAyM',
  'type': 'file',
  'size': 1481},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/22012610182_26af763d9e_o.jpg',
  'CID': 'QmQSrnB6Tvs7WvGFx7t3HcmRka6F3ZQFZWWdwJbY79MafL',
  'type': 'file',
  'size': 311358},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/22012610182_

In [25]:
# Given a directory CID to cat
fs.ipfs.cat('QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8', recursive=True).keys() 

dict_keys(['QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21751523890_3b65b9f030_o.jpg', 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21751523890_3b65b9f030_o.jpg.xml', 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21878909302_a369c0c4de_o.jpg', 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21878909302_a369c0c4de_o.jpg.xml', 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/22012610182_26af763d9e_o.jpg', 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/22012610182_26af763d9e_o.jpg.xml', 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/_meta.xml'])

## Ls

#### LS Remote File - CID Path 


In [26]:
fs.ipfs.ls('QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8')

[{'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21751523890_3b65b9f030_o.jpg',
  'CID': 'QmbgeZnTVXAhW2CS15cjo981P81kULbuUqJ672YEGZUqiM',
  'type': 'file',
  'size': 386602},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21751523890_3b65b9f030_o.jpg.xml',
  'CID': 'QmRfG6WQEBXGJge8EQVXbYfYBS25pmhLoaczd5CQxjcT4U',
  'type': 'file',
  'size': 3513},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21878909302_a369c0c4de_o.jpg',
  'CID': 'QmXfbZ7H946MeecTWZqcdWKnPwudcqcokTFctJ5LeqMDK3',
  'type': 'file',
  'size': 1130403},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/21878909302_a369c0c4de_o.jpg.xml',
  'CID': 'QmWxfUFH6ka7P5kMfw3ti3FhRqBpbkgFFY3NyaCZYxbAyM',
  'type': 'file',
  'size': 1481},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/22012610182_26af763d9e_o.jpg',
  'CID': 'QmQSrnB6Tvs7WvGFx7t3HcmRka6F3ZQFZWWdwJbY79MafL',
  'type': 'file',
  'size': 311358},
 {'name': 'QmQwhnitZWNrVQQ1G8rL4FRvvZBUvHcxCtUreskfnBzvD8/22012610182_

#### LS Remote Folder - CID Path

## Get

#### GET CID File

In [27]:
files = glob.glob('test/data/output/*')
for f in files:
    os.remove(f)
    
    
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*')])
fs.ipfs.get(rpath='QmbgeZnTVXAhW2CS15cjo981P81kULbuUqJ672YEGZUqiM', 
            lpath='test/data/output/test.png', # a filename must be given
            recursive=True, 
            return_cid=False)

print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*')])

Before:  []
After:  ['test/data/output/test.png']


#### Get Remote Folder - CID Path

In [28]:

files = glob.glob('test/data/output/*')
for f in files:
    os.remove(f)
    
    
fs.ipfs.change_gateway_type = 'local'
cid = fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
fs.ipfs.change_gateway_type = 'public'
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

fs.ipfs.get(rpath=cid, lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

Changed to local node
Changed to public node
Before:  []
After:  ['test/data/output/bruh.txt', 'test/data/output/yo.txt']
